In [ ]:
# !pip install matplotlib

# XMR Control Analysis in Temperature Tolarant Systems

## Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
import os

# Move the current working directory to SignalWatch root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))

# Add the project root to the sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt

from control.control_chart import ControlChart

## Spark Session Creation

In [ ]:
# New Spark Session
spark = SparkSession.builder.appName('ControlAnalysis').getOrCreate()

In [ ]:
# Read the Parquet file into a Spark DataFrame
df = spark.read.parquet('./scripts/data/parquet/temperature_data_high.parquet')

In [ ]:
# Example Usage:
# Assuming df is a PySpark DataFrame with 'Index' as the time column and 'Temp_F' as the value column
chart = ControlChart(df, 'Temp_F', 'Index')

In [ ]:
## First iteration of control check
first_result_df = chart.run_first_control_chart(chart_type='xmr')
# first_result_df.show()

## Visualize First Iteration of Process Analysis

In [ ]:
# Converting to DataFrame
first_result_pdf = first_result_df.toPandas()
# df = pd.DataFrame(data)

# Plotting the XMR chart
plt.figure(figsize=(12, 6))
plt.plot(first_result_pdf['Index'], first_result_pdf['Temp_F'], marker='o', label='Temperature (F)')
plt.axhline(y=first_result_pdf['UCL'][0], color='r', linestyle='--', label='UCL')
plt.axhline(y=first_result_pdf['LCL'][0], color='r', linestyle='--', label='LCL')
plt.axhline(y=first_result_pdf['CL'][0], color='g', linestyle='-', label='CL')

# Highlighting out-of-control points
out_of_control_points = first_result_pdf[first_result_pdf['out_of_control'] == True]
plt.scatter(out_of_control_points['Index'], out_of_control_points['Temp_F'], color='red', label='Out of Control', zorder=5)

plt.title('XMR Control Chart')
plt.xlabel('Index')
plt.ylabel('Temperature (F)')
plt.legend()
plt.grid(True)
plt.show()

## Complete control analysis

In [ ]:
# complete control analysis
control_result_df, removed_point_df = chart.run_control_chart(chart_type='xmr')

## Control Chart

In [ ]:
# Converting to DataFrame
control_result_pdf = control_result_df.toPandas()
# df = pd.DataFrame(data)

# Plotting the XMR chart
plt.figure(figsize=(12, 6))
plt.plot(control_result_pdf['Index'], control_result_pdf['Temp_F'], marker='o', label='Temperature (F)')
plt.axhline(y=control_result_pdf['UCL'][0], color='r', linestyle='--', label='UCL')
plt.axhline(y=control_result_pdf['LCL'][0], color='r', linestyle='--', label='LCL')
plt.axhline(y=control_result_pdf['CL'][0], color='g', linestyle='-', label='CL')

# Highlighting out-of-control points
out_of_control_points = control_result_pdf[control_result_pdf['out_of_control'] == True]
plt.scatter(out_of_control_points['Index'], out_of_control_points['Temp_F'], color='red', label='Out of Control', zorder=5)

plt.title('XMR Control Chart')
plt.xlabel('Index')
plt.ylabel('Temperature (F)')
plt.legend()
plt.grid(True)
plt.show()

## Data Points Omitted from Control Chart Analysis

In [ ]:
# Converting to DataFrame
removed_point_df = removed_point_df.orderBy('Index')
removed_point_pdf = removed_point_df.toPandas()
# df = pd.DataFrame(data)

# Plotting the XMR chart
plt.figure(figsize=(12, 6))
# plt.plot(removed_point_pdf['Index'], removed_point_pdf['Temp_F'], marker='o', label='Temperature (F)')

# Control limits from complete control analysis 
plt.axhline(y=control_result_pdf['UCL'][0], color='r', linestyle='--', label='UCL')
plt.axhline(y=control_result_pdf['LCL'][0], color='r', linestyle='--', label='LCL')
plt.axhline(y=control_result_pdf['CL'][0], color='g', linestyle='-', label='CL')


# Highlighting out-of-control points
out_of_control_points = removed_point_pdf[removed_point_pdf['out_of_control'] == True]
plt.scatter(out_of_control_points['Index'], out_of_control_points['Temp_F'], color='red', label='Out of Control', zorder=5)

plt.title('XMR Control Chart')
plt.xlabel('Index')
plt.ylabel('Temperature (F)')
plt.legend()
plt.grid(True)
plt.show()